# NAT Traversal 

The following is our implementation of NAT traversal, inheriting the Minimal class from `minimal.py`.

To make the code work with Jupyter Notebook, we had to make some small adjustments: As you can't call a python module with command line arguments from Jupyter, we had to specify `-n` manually. The argument options for printing the public endpoint and NAT type and printing the available audio devices are also added before declaring the `NAT_traversal` class.

In [1]:
from minimal import Minimal, int_or_str
import os
import sys
import stun
import argparse
import sounddevice as sd

sys.argv = ['NAT_traversal.py', '-n']

parser = argparse.ArgumentParser()
parser.add_argument("-n", "--get-endpoint-nat", action="store_true", help="Print public endpoint and NAT type")
parser.add_argument("-d", "--list-devices", action="store_true", help="Print the available audio devices and quit")

_StoreTrueAction(option_strings=['-d', '--list-devices'], dest='list_devices', nargs=0, const=True, default=False, type=None, choices=None, required=False, help='Print the available audio devices and quit', metavar=None)

The class `NAT_traversal` inherits from `minimal.Minimal` and additionaly contains our (very simple) way of figuring out our external IP and port and the NAT type we are behind. 

In [2]:
class NAT_traversal(Minimal):
    def __init__(self):
        super().__init__()
        self.nat_type, self.external_ip, self.external_port = stun.get_ip_info(stun_host='stun.l.google.com')
        self.our_nat_type = self.get_nat_type()

    def get_nat_type(self):
        nat_type1, external_ip1, external_port1 = stun.get_ip_info(stun_host='stun.l.google.com')
        nat_type2, external_ip2, external_port2 = stun.get_ip_info(stun_host='stun.l.google.com')

        if external_port1 == external_port2:
            return "Cone"
        
        else:
            return "Symmetric"

Subsequently, we parse the command line arguments (just `-n` in this case), initiate a `NAT_traversal` instance and either print our NAT type and public IP and port, our available sound devices or run InterCom.

In [3]:
args = parser.parse_args()

n = NAT_traversal()

if args.get_endpoint_nat:
    print("NAT type (self-determined): " + n.our_nat_type)
    print("NAT type (STUN servers): " + n.nat_type)
    print("Public IP: " + n.external_ip)
    print("Port: " + str(n.external_port))
    
#elif args.list_devices:
#    print("Available devices:")
#    print(sd.query_devices())

else:
    try:
        n.run()
    except KeyboardInterrupt:
        parser.exit("\nSIGINT received")
    finally:
        n.print_final_averages()

(INFO) minimal: A minimal InterCom (no compression, no quantization, no transform, ... only provides a bidirectional (full-duplex) transmission of raw (playable) chunks. 
(INFO) minimal: NUMBER_OF_CHANNELS = 2
(INFO) minimal: chunk_time = 0.023219954648526078 seconds


NAT type (self-determined): Cone
NAT type (STUN servers): Full Cone
Public IP: 150.214.223.70
Port: 54320
